In [1]:
import geocoder
import pandas as pd
import requests

In [2]:
g = geocoder.osm('San Lorenzo, Ilocos Norte')

Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=San+Lorenzo%2C+Ilocos+Norte&format=jsonv2&addressdetails=1&limit=1


In [3]:
g.json

In [4]:
df = pd.read_excel('Output/Updated_RDO No. 1 - Laoag City, Ilocos Norte.xlsx')

In [5]:
df.iloc[0]

Province                                               ILOCOS NORTE
City/Municipality                                        LAOAG CITY
Barangay                                    SAN LORENZO (POBLACION)
Street/Subdivision      DONA MARIA G. CASTRO ST (BRILLANTES STREET)
Vicinity              D SAMONTE ST. - PRIMO LAZARO ST(P. BURGOS ST)
Classification                                                   RR
ZV/SQM                                                         7300
Name: 0, dtype: object

In [6]:
df

,Province,City/Municipality,Barangay,Street/Subdivision,Vicinity,Classification,ZV/SQM
0,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - PRIMO LAZARO ST(P. BURGOS ST),RR,7300
1,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST,CR*,8700
2,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST,RR,7300
3,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - RIZAL ST,RR,7300
4,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),INTERIOR,RR,5500
...,...,...,...,...,...,...,...
14707,ILOCOS NORTE,VINTAR,ISIC-ISIC,ALL LOTS,ALONG THE BORDER OF THE,RR,190
14708,ILOCOS NORTE,VINTAR,ISIC-ISIC,ALL LOTS,MOUNTAIN,A3,48
14709,ILOCOS NORTE,VINTAR,ISIC-ISIC,ALL LOTS,MOUNTAIN,A15,53
14710,ILOCOS NORTE,VINTAR,ISIC-ISIC,ALL LOTS,MOUNTAIN,A16*,63


In [7]:
g = geocoder.osm('New York city')

Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=New+York+city&format=jsonv2&addressdetails=1&limit=1


In [11]:
import requests

address = 'AMAPOLA, BEL-AIR, MAKATI CITY, NCR, Philippines'
url = f"http://photon.komoot.io/api/?q={address}"

# Send request to Photon API
response = requests.get(url).json()

# Extract latitude and longitude
if response["features"]:
    location = response["features"][0]["geometry"]["coordinates"]
    print(f"Latitude: {location[1]}, Longitude: {location[0]}")
    print(response)
else:
    print("Location not found")


Latitude: 14.5628316, Longitude: 121.0389691
{'features': [{'geometry': {'coordinates': [121.0389691, 14.5628316], 'type': 'Point'}, 'type': 'Feature', 'properties': {'osm_id': 655807603, 'extent': [121.038921, 14.562844, 121.038984, 14.562824], 'country': 'Philippines', 'city': 'Makati', 'countrycode': 'PH', 'postcode': '1209', 'locality': 'Bel-Air', 'county': 'Southern Manila District', 'type': 'street', 'osm_type': 'W', 'osm_key': 'highway', 'district': 'District I', 'osm_value': 'residential', 'name': 'Amapola Street', 'state': 'Metro Manila'}}, {'geometry': {'coordinates': [121.02830105627338, 14.56517335], 'type': 'Point'}, 'type': 'Feature', 'properties': {'osm_id': 368314130, 'extent': [121.0281167, 14.56535, 121.0284854, 14.5649966], 'country': 'Philippines', 'city': 'Makati', 'countrycode': 'PH', 'postcode': '1210', 'locality': 'Century City', 'county': 'Southern Manila District', 'type': 'house', 'osm_type': 'W', 'osm_key': 'building', 'street': 'Kalayaan Avenue', 'district'

In [9]:
response["features"][0]["geometry"]

{'coordinates': [120.5939089, 18.1988381], 'type': 'Point'}

In [39]:
import requests
import pandas as pd

# Initialize variables to store last known Street/Subdivision and coordinates
last_street = None
last_latitude = None
last_longitude = None

def get_coordinates(row):
    global last_street, last_latitude, last_longitude
    
    # Get the current street/subdivision value
    current_street = row['Street/Subdivision']

    # Check if current street/subdivision matches the last one processed
    if current_street == last_street:
        # Return last known latitude and longitude without making an API call
        print("NO API")
        return pd.Series([last_latitude, last_longitude])

    # Construct address combinations based on specificity
    address_combinations = [
        f"{row['Street/Subdivision']}, {row['Barangay']}, {row['City/Municipality']}, {row['Province']}",
        f"{row['Street/Subdivision']}, {row['City/Municipality']}, {row['Province']}",
        f"{row['Barangay']}, {row['City/Municipality']}, {row['Province']}",
        f"{row['City/Municipality']}, {row['Province']}",
        f"{row['Province']}"
    ]

    # If the street/subdivision is new, attempt to fetch coordinates
    for address in address_combinations:
        if not pd.isnull(address) and address.strip():  # Ensure address is not empty
            try:
                # Send a request to Photon API
                response = requests.get(
                    'https://photon.komoot.io/api/',
                    params={'q': address, 'limit': 1}
                )
                data = response.json()
                
                # Check if any result is found
                if data.get('features'):
                    location = data['features'][0]['geometry']['coordinates']
                    latitude, longitude = location[1], location[0]  # Photon returns [longitude, latitude]
                    
                    # Update last known street/subdivision and coordinates
                    last_street = current_street
                    last_latitude, last_longitude = latitude, longitude
                    
                    # Return the latitude and longitude
                    return pd.Series([latitude, longitude])
            except Exception as e:
                print(f"Error fetching coordinates for address '{address}': {e}")
                continue
    
    # If no valid coordinates found, return None
    return pd.Series([None, None])


In [13]:
df = pd.read_excel("Output/Updated_RDO No. 1 - Laoag City, Ilocos Norte.xlsx")

In [14]:
df.iloc[1]

Province                                             ILOCOS NORTE
City/Municipality                                      LAOAG CITY
Barangay                                  SAN LORENZO (POBLACION)
Street/Subdivision    DONA MARIA G. CASTRO ST (BRILLANTES STREET)
Vicinity                   PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST
Classification                                                CR*
ZV/SQM                                                       8700
Name: 1, dtype: object

In [15]:

# Load your Excel file into a DataFrame



# Apply the function to each row and create 'Latitude' and 'Longitude' columns
df[['Latitude', 'Longitude']] = df.apply(get_coordinates, axis=1)

# Save the updated DataFrame to a new Excel file
df.to_excel("updated_with_coordinates.xlsx", index=False)


Latitude: 18.20392195, Longitude: 120.58498964258703


KeyboardInterrupt: 

In [40]:
df2 = df.head(100)
df2

,Province,City/Municipality,Barangay,Street/Subdivision,Vicinity,Classification,ZV/SQM
0,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - PRIMO LAZARO ST(P. BURGOS ST),RR,7300
1,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST,CR*,8700
2,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST,RR,7300
3,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - RIZAL ST,RR,7300
4,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),INTERIOR,RR,5500
...,...,...,...,...,...,...,...
95,ILOCOS NORTE,LAOAG CITY,SANTA JOAQUINA (POBLACION),ALL OTHER STREETS,ALONG THE ROAD,A50,2700
96,ILOCOS NORTE,LAOAG CITY,SANTA JOAQUINA (POBLACION),ALL OTHER STREETS,INTERIOR,RR,4000
97,ILOCOS NORTE,LAOAG CITY,SANTA JOAQUINA (POBLACION),ALL OTHER STREETS,INTERIOR,A1*,2600
98,ILOCOS NORTE,LAOAG CITY,SANTA JOAQUINA (POBLACION),ALL OTHER STREETS,INTERIOR,A2*,2400


In [41]:
len(list(df['Street/Subdivision'].unique()))

736

In [42]:
df2[['Latitude', 'Longitude']] = df2.apply(get_coordinates, axis=1)

# Save the updated DataFrame to a new Excel file
df2.to_excel("updated_with_coordinates.xlsx", index=False)

NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API
NO API


C:\Users\oyutn\AppData\Local\Temp\ipykernel_2740\1889850206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['Latitude', 'Longitude']] = df2.apply(get_coordinates, axis=1)
C:\Users\oyutn\AppData\Local\Temp\ipykernel_2740\1889850206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['Latitude', 'Longitude']] = df2.apply(get_coordinates, axis=1)


In [43]:
df2

,Province,City/Municipality,Barangay,Street/Subdivision,Vicinity,Classification,ZV/SQM,Latitude,Longitude
0,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - PRIMO LAZARO ST(P. BURGOS ST),RR,7300,18.203922,120.584990
1,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST,CR*,8700,18.203922,120.584990
2,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),PRIMO LAZARO ST(P BURGOS ST)- RIZAL ST,RR,7300,18.203922,120.584990
3,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),D SAMONTE ST. - RIZAL ST,RR,7300,18.203922,120.584990
4,ILOCOS NORTE,LAOAG CITY,SAN LORENZO (POBLACION),DONA MARIA G. CASTRO ST (BRILLANTES STREET),INTERIOR,RR,5500,18.203922,120.584990
...,...,...,...,...,...,...,...,...,...
95,ILOCOS NORTE,LAOAG CITY,SANTA JOAQUINA (POBLACION),ALL OTHER STREETS,ALONG THE ROAD,A50,2700,14.661238,121.022708
96,ILOCOS NORTE,LAOAG CITY,SANTA JOAQUINA (POBLACION),ALL OTHER STREETS,INTERIOR,RR,4000,14.661238,121.022708
97,ILOCOS NORTE,LAOAG CITY,SANTA JOAQUINA (POBLACION),ALL OTHER STREETS,INTERIOR,A1*,2600,14.661238,121.022708
98,ILOCOS NORTE,LAOAG CITY,SANTA JOAQUINA (POBLACION),ALL OTHER STREETS,INTERIOR,A2*,2400,14.661238,121.022708


In [45]:
import os
import pandas as pd

# Define the directory where your Excel files are stored
directory_path = 'Output'

# Initialize a list to store the results as dictionaries
data = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    # Only process Excel files
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        # Create the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Read the Excel file
        try:
            df = pd.read_excel(file_path)
            
            # Get the number of rows
            num_rows = len(df)
            
            # Count unique values in the "Streets/Subdivisions" column
            if "Street/Subdivision" in df.columns:
                unique_values = df["Street/Subdivision"].nunique()
            else:
                unique_values = None
                print(f"Column 'Street/Subdivision' not found in file: {filename}")
            
            # Append the results as a dictionary to the list
            data.append({
                "filename": filename,
                "num_rows": num_rows,
                "unique_streets_subdivisions": unique_values
            })
        
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(data)

# Display the DataFrame
print(results_df)


Error processing file ~$Updated_RDO No. 24 - Valenzuela City.xlsx: [Errno 13] Permission denied: 'Output\\~$Updated_RDO No. 24 - Valenzuela City.xlsx'
                                              filename  num_rows  \
0    Updated_RDO No. 1 - Laoag City, Ilocos Norte.xlsx     14712   
1       Updated_RDO No. 10 - Bontoc, Mt. Province.xlsx     19075   
2    Updated_RDO No. 100 - Ozamiz City, Misamis Occ...      2581   
3    Updated_RDO No. 101 - Iligan City, Lanao del N...     18338   
4    Updated_RDO No. 102 - Marawi City, Lanao del S...     24191   
..                                                 ...       ...   
117               Updated_RDO No. 95 - Jolo, Sulu.xlsx       437   
118        Updated_RDO No. 96 - Bongao, Tawi-Tawi.xlsx       363   
119  Updated_RDO No. 97 - Gingoog City, Misamis Ori...      1419   
120  Updated_RDO No. 98 - Cagayan de Oro City, Misa...      7785   
121  Updated_RDO No. 99 - Malaybalay City, Bukidnon...     35797   

     unique_streets_subdivisions

In [46]:
results_df

,filename,num_rows,unique_streets_subdivisions
0,"Updated_RDO No. 1 - Laoag City, Ilocos Norte.xlsx",14712,735
1,"Updated_RDO No. 10 - Bontoc, Mt. Province.xlsx",19075,18
2,"Updated_RDO No. 100 - Ozamiz City, Misamis Occ...",2581,234
3,"Updated_RDO No. 101 - Iligan City, Lanao del N...",18338,416
4,"Updated_RDO No. 102 - Marawi City, Lanao del S...",24191,239
...,...,...,...
117,"Updated_RDO No. 95 - Jolo, Sulu.xlsx",437,1
118,"Updated_RDO No. 96 - Bongao, Tawi-Tawi.xlsx",363,21
119,"Updated_RDO No. 97 - Gingoog City, Misamis Ori...",1419,83
120,"Updated_RDO No. 98 - Cagayan de Oro City, Misa...",7785,1085


In [49]:
import os
import pandas as pd

# Define the directory where your Excel files are stored
directory_path = 'Output'

# Initialize a list to store the results as dictionaries
data = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    # Only process Excel files
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        # Create the full file path
        file_path = os.path.join(directory_path, filename)
        
        # Read the Excel file
        try:
            df = pd.read_excel(file_path)
            
            # Get the number of rows
            num_rows = len(df)
            
            # Count unique values in specified columns
            unique_streets = df["Street/Subdivision"].nunique() if "Street/Subdivision" in df.columns else None
            unique_barangay = df["Barangay"].nunique() if "Barangay" in df.columns else None
            unique_city_municipality = df["City/Municipality"].nunique() if "City/Municipality" in df.columns else None
            
            # Append the results as a dictionary to the list
            data.append({
                "filename": filename,
                "num_rows": num_rows,
                "unique_streets_subdivisions": unique_streets,
                "unique_barangay": unique_barangay,
                "unique_city_municipality": unique_city_municipality
            })
        
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(data)

# Display the DataFrame
print(results_df)


Error processing file ~$Updated_RDO No. 24 - Valenzuela City.xlsx: [Errno 13] Permission denied: 'Output\\~$Updated_RDO No. 24 - Valenzuela City.xlsx'
                                              filename  num_rows  \
0    Updated_RDO No. 1 - Laoag City, Ilocos Norte.xlsx     14712   
1       Updated_RDO No. 10 - Bontoc, Mt. Province.xlsx     19075   
2    Updated_RDO No. 100 - Ozamiz City, Misamis Occ...      2581   
3    Updated_RDO No. 101 - Iligan City, Lanao del N...     18338   
4    Updated_RDO No. 102 - Marawi City, Lanao del S...     24191   
..                                                 ...       ...   
117               Updated_RDO No. 95 - Jolo, Sulu.xlsx       437   
118        Updated_RDO No. 96 - Bongao, Tawi-Tawi.xlsx       363   
119  Updated_RDO No. 97 - Gingoog City, Misamis Ori...      1419   
120  Updated_RDO No. 98 - Cagayan de Oro City, Misa...      7785   
121  Updated_RDO No. 99 - Malaybalay City, Bukidnon...     35797   

     unique_streets_subdivisions

In [47]:
# Sum of 'num_rows' column
total_rows = results_df['num_rows'].sum()

# Sum of 'unique_streets_subdivisions' column
total_unique_streets = results_df['unique_streets_subdivisions'].sum()

print(f"Total Rows: {total_rows}")
print(f"Total Unique 'Streets/Subdivisions': {total_unique_streets}")


Total Rows: 762619
Total Unique 'Streets/Subdivisions': 67313


In [48]:
results_df.describe()

,num_rows,unique_streets_subdivisions
count,122.000000,122.000000
mean,6250.975410,551.745902
std,6155.529237,486.821901
min,0.000000,0.000000
25%,2045.250000,204.750000
50%,3975.000000,459.500000
75%,8606.000000,779.750000
max,35797.000000,2608.000000


In [51]:

total_rows = results_df['num_rows'].sum()
total_unique_streets = results_df['unique_streets_subdivisions'].sum()
total_unique_barangays = results_df['unique_barangay'].sum()
total_unique_cities = results_df['unique_city_municipality'].sum()

print(f"Total Rows: {total_rows}")
print(f"Total Unique 'Street/Subdivision': {total_unique_streets}")
print(f"Total Unique 'Barangay': {total_unique_barangays}")
print(f"Total Unique 'City/Municipality': {total_unique_cities}")


Total Rows: 762619
Total Unique 'Street/Subdivision': 67313
Total Unique 'Barangay': 27843
Total Unique 'City/Municipality': 1576
